In [1]:
from bs4 import BeautifulSoup  

In [ ]:
import random
import scrapy
import csv
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup, Tag
import time
def print_css_tree(html):
    soup = BeautifulSoup(html, 'html.parser')

    def recursive_print(tag, prefix="."):
        if isinstance(tag, Tag):  # Check if tag is a BeautifulSoup Tag object
            class_str = ".".join(tag.get('class', []))
            print(f"{prefix}{tag.name}.{class_str}")
            prefix += "  "
            for child in tag.children:
                recursive_print(child, prefix)
                
    recursive_print(soup)



def checkProfile(row,profiles):
    try:
        for sublist in profiles:
            if row in sublist:
                return True
        return False
    except:
        return False

def convert_to_number(s):
            multipliers = {'K': 1000, 'M': 1000000, 'B': 1000000000}
            # Check the last character of the string
            if s[-1] in multipliers:
                # If the last character is a multiplier, remove it from the string,
                # convert the remaining string to a float, multiply by the appropriate value,
                # and convert the result to an integer
                return int(float(s[:-1]) * multipliers[s[-1]])
            else:
                # If the last character is not a multiplier, just convert the string to an integer
                try:
                    return int(s)
                except ValueError:
                    try:
                        return float(s)
                    except ValueError:
                        return s
            
            
def extract_values(text):
    # Find the index of the item with '%'
    index = next((i for i, s in enumerate(text) if '%' in s), None)
    if index is not None and index > 0:
        # If found and it's not the first item, return the item and the one before it
        return [text[index - 1], text[index]]
    else:
        # If not found or it's the first item, return None
        return [None, None]

# Usage:            
class BlogSpider(scrapy.Spider):  
    
    def __init__(self):
  
        self.name = 'blogspider'
        self.base= 'https://vidiq.com/youtube-stats/channel/'
        self.start_urls =[]
        self.profiles = []
        
        with open('data\\profiles\\profiles2024-10.csv', 'r', encoding='utf-8') as file:
            reader = csv.reader(file)
            profiles = list(reader)


        with open('data\\profiles\\profilesids2024-10.csv', 'r', encoding='utf-8') as file:
            reader = csv.reader(file)
            data = list(reader)
            print(data[0])
            for row in data:
                print(row)
                booler = checkProfile(row[0],profiles)
                if not booler:
                    self.start_urls.append(self.base+(row[0]))

    
    def parse(self, response):
        time.sleep(12)
        time.sleep(random.randint(1, 4))
        # Extract data from the new CSS selector
        URL = response.url
        page= URL.rsplit('/')[-2]
        currentAccount = response.css('h1::text').get()
        element = response.xpath('/html/body/main/section/div/div[2]/div[2]/div[2]/div[1]/div' )
        time.sleep(1)
        subscribers = element.css('div:nth-child(2)>div>div>p:nth-child(2)::text').get()
        subscribersChange = element.css('div:nth-child(2)>div>div>p:nth-child(3)>*::text').getall()
        view = element.css('div:nth-child(3)>div>div>p:nth-child(2)::text').get()
        viewChange = element.css('div:nth-child(3)>div>div>p:nth-child(3)>*::text').getall()
        earnings = element.css('div:nth-child(4)>div>div>p::text').getall()
        engagment = element.css('div:nth-child(5)>div>div>p::text').get()
        uploadFrequency = element.css('div:nth-child(6)>div>div>p::text').get()
        avgLength = element.css('div:nth-child(7)>div>div>p::text').get()
        trendelement= response.xpath('/html/body/main/section/div/div[2]/div[2]/div[2]/div[2]/div/div/div[1]/table/tbody')
        performanceData = []

        # Iterate over each <tr> in the table body
        for tr in trendelement.xpath('.//tr'):
            # Extract the text from each <span> within the <td> elements
            row_data = [td.xpath('.//span/text()').getall() for td in tr.xpath('.//td')]
            # Append the row data to the list
            performanceData.append(row_data)
    
        try:
            subscribers = convert_to_number(subscribers)
        except:
            subscribers = convert_to_number(subscribers)
        view = convert_to_number(view)
        earningsLow = convert_to_number(earnings[1])
        earninghigh = convert_to_number(earnings[3])         
        yield self.addProfile(currentAccount,subscribers,''.join(subscribersChange[:2]),view,''.join(viewChange[:2]),earningsLow, earninghigh,engagment,uploadFrequency,avgLength,page,performanceData)

    
    
    def addProfile(self,currentAccount,suscribers,suscribersChange,view,viewChange,earningsLow, earninghigh,engagment,uploadFrequency,avgLength,page,performanceData):
        with open('data\\profiles\\profiles2024-10.csv', 'a',newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([currentAccount,suscribers,suscribersChange,view,viewChange,earningsLow, earninghigh,engagment,uploadFrequency,avgLength,page,performanceData])



if __name__ == "__main__":
    process = CrawlerProcess({
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
    })
    import csv
    import pandas as pd

    # Load the CSV file into a DataFrame
    df = pd.read_csv('data\\profiles\\profiles2024-10.csv')

    # Drop duplicate rows
    df = df.drop_duplicates()

    # Write the DataFrame back to the CSV file
    df.to_csv('data\\profiles\\profiles2024-10.csv', index=False)
    headers = ['currentAccount', 'suscribers', 'suscribersChange', 'view', 'viewChange', 'earningsLow', 'earninghigh', 'engagment', 'uploadFrequency', 'avgLength','page']

    with open('data\\profiles\\profiles2024-10.csv', 'r+', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        first_row = next(reader, None)
        if first_row != headers:
            # Move the file pointer to the beginning of the file
            file.seek(0)
            # Create a writer object
            writer = csv.writer(file)
            # Write the headers
            writer.writerow(headers)
            # Write the original first row if it exists
            if first_row:
                writer.writerow(first_row)
    process.crawl(BlogSpider)
    process.start() # the script will block here until the crawling is finished



2024-11-07 21:38:52 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-11-07 21:38:52 [scrapy.utils.log] INFO: Versions: lxml 5.2.2.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.3.0, Python 3.12.3 | packaged by Anaconda, Inc. | (main, May  6 2024, 19:42:21) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 24.1.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.7, Platform Windows-11-10.0.22631-SP0
2024-11-07 21:38:53 [scrapy.addons] INFO: Enabled addons:
[]
2024-11-07 21:38:53 [py.warnings] WARNING: c:\Users\Abdul\miniconda3\envs\imggen\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.



['UCuVPpxrm2VAgpH3Ktln4HXg', 'UCuVPpxrm2VAgpH3Ktln4HXg']
['UCuVPpxrm2VAgpH3Ktln4HXg', 'UCuVPpxrm2VAgpH3Ktln4HXg']
['UC_n7zvkh3y3PLBIUA1lHkjw', 'UC_n7zvkh3y3PLBIUA1lHkjw']
['UCn3AViOfcFN4zIICMC5hvPg', '@shilohandbros']
['UC_q5WZtFp36adwqhKpZzxwQ', '@SypherPK']
['UC1E1SVcVyU3ntWMSQEp38Yw', '@TheProfGShow']
['UCYzEMRKqrh01-tauv7MYyVQ', '@MSA.official']
['UCJjSDX-jUChzOEyok9XYRJQ', '@jubilee']
['UC7u9o8BHiJyH2_cef_nC7tQ', '@DentalDigest']
['UCqFzWxSCi39LnW1JKFR3efg', '@SaturdayNightLive']
['UCMiJRAwDNSNzuYeN2uWa0pA', '@Mrwhosetheboss']
['UCqVCSWnuOSPm26orvNDTuWQ', '@TGplays']
['UCJcQeT8LADOXRROXCGM4dRg', '@TheCrazyGorilla']
['UCI0J7D4v99_A7DWbyaQc57Q', '@shameless']
['UClHbjVXWB1rCNfB-MQDw-Nw', '@KoreanComic']
['UCX6OQ3DkcsbYNE6H8uQQuVA', '@MrBeast']
['UCupvZG-5ko_eiXAupbDfxWw', '@CNN']
['UCDogdKl7t7NHzQ95aEwkdMw', '@Sidemen']
['UCQIUhhcmXsu6cN6n3y9-Pww', '@Jesser']
['UCJHAT3Uvv-g3I8H3GhHWV7w', '@RyanHallYall']
['UCO0akufu9MOzyz3nvGIXAAw', '@SkyNewsAustralia']
['UCzEy7pi3B7TIS9cn_sdKK9A', 

2024-11-07 21:38:54 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2024-11-07 21:38:54 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2024-11-07 21:38:54 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.dow

2024-06-19 15:25:30 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-06-19 15:25:30 [scrapy.utils.log] INFO: Versions: lxml 5.2.2.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.3.0, Python 3.12.3 | packaged by Anaconda, Inc. | (main, May  6 2024, 19:42:21) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 24.1.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.7, Platform Windows-11-10.0.22631-SP0
2024-06-19 15:25:30 [scrapy.addons] INFO: Enabled addons:
[]
2024-06-19 15:25:30 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2024-06-19 15:25:30 [scrapy.extensions.telnet] INFO: Telnet Password: 9654cf1136c0d3e0
2024-06-19 15:25:30 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2024-06-19 15:25:30 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windo

['UC1YrFAk8SjTJIvKqLHHTaow', 'UC1YrFAk8SjTJIvKqLHHTaow']
['UCnr1-8gbgQ7YFd8twlBDhhg', 'UCnr1-8gbgQ7YFd8twlBDhhg']
['UCMwJJL5FJFuTRT55ksbQ4GQ', '@AsmongoldClips']


ReactorNotRestartable: 